# Hotspot data <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with the `NCI`|`DEA Sandbox` environment only
* **Products used:** 
[s2a_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), 
[s2b_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule),
[ga_ls5t_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
[ga_ls7e_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
[ga_ls8c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3)
* **Special requirements:** An _optional_ description of any special requirements, e.g. If running on the [NCI](https://nci.org.au/), ensure that `module load otps` is run prior to launching this notebook
* **Prerequisites:** An _optional_ list of any notebooks that should be run or content that should be understood prior to launching this notebook


## Background
An *optional* overview of the scientific, economic or environmental management issue or challenge being addressed by Digital Earth Australia. 
For `Beginners_Guide` or `Frequently_Used_Code` notebooks, this may include information about why the particular technique or approach is useful or required. 
If you need to cite a scientific paper or link to a website, use a persistent DOI link if possible and link in-text (e.g. [Dhu et al. 2017](https://doi.org/10.1080/20964471.2017.1402490)).

## Description
A _compulsory_ description of the notebook, including a brief overview of how Digital Earth Australia helps to address the problem set out above.
It can be good to include a run-down of the tools/methods that will be demonstrated in the notebook:

1. First we do this
2. Then we do this
3. Finally we do this

***

## Getting started

Provide any particular instructions that the user might need, e.g. To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Scripts` directory.

In [3]:
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt


## Analysis

In [ ]:
import zipfile
import os

zip_name = 'hotspots.zip'
output_dir = ''

# Extract into output_dir
with zipfile.ZipFile(zip_name, 'r') as zip_ref:        
    zip_ref.extractall(output_dir)        
    print(f'Unzipping output files to: '
          f'{output_dir if output_dir else os.getcwd()}')

In [5]:
# clip_extent = gpd.read_file('clip_extent.geojson')
# hotspots_gdf = gpd.read_file('hotspots.json').cx[144.07:154.42, -26.07:-39.21]

DriverError: clip_extent.geojson: No such file or directory

In [53]:
hotspots_subset = hotspots_gdf
hotspots_subset = hotspots_subset[hotspots_subset.satellite != 'NOAA 19']
hotspots_subset = hotspots_subset.loc[hotspots_subset.confidence > 49]
hotspots_subset = hotspots_subset[hotspots_subset.within(clip_extent.unary_union)]
hotspots_subset['datetime'] = pd.to_datetime(hotspots_subset['start_dt'])
hotspots_subset['hours_since_hotspot'] = (((hotspots_subset['datetime'].max() - hotspots_subset['datetime'])).astype('timedelta64[m]') / 60)
hotspots_subset = hotspots_subset.loc[:,['datetime', 'latitude', 'longitude', 'hours_since_hotspot', 'confidence', 'geometry']]
hotspots_subset.sort_values('datetime', ascending=True, inplace=True)
# pd.DataFrame(hotspots_subset).to_csv('hotspots.csv')

In [34]:
import datacube
dc = datacube.Datacube(app="DEA_hotspots")

# Create a query object
query = {
    "time": '2018-01-01',
    "x": (144.07, 154.42),
    "y": (-26.07, -39.21),
#     "x": (144.07, 144.09),
#     "y": (-26.07, -26.09),
    "measurements": ["red", "green", "blue"],
    "output_crs": "EPSG:4326",
    "resolution": (-0.03, 0.03),
}

ds = dc.load(product="ls8_nbart_geomedian_annual", **query)


In [103]:
import datacube
dc = datacube.Datacube(app="DEA_hotspots")

# Create a query object
query = {
    "time": '2018',
    "x": (144.07, 144.09),
    "y": (-26.07, -26.09),
    "measurements": ["red", "green", "blue"],
    "output_crs": "EPSG:4326",
    "resolution": (-0.03, 0.03),
}

ds = dc.load(product="ls8_nbart_geomedian_annual", **query)
ds

<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 2018-01-01
  * latitude   (latitude) float64 -26.09
  * longitude  (longitude) float64 144.1
Data variables:
    red        (time, latitude, longitude) int16 1576
    green      (time, latitude, longitude) int16 967
    blue       (time, latitude, longitude) int16 572
Attributes:
    crs:      EPSG:4326

In [37]:
comp_dates[0:5]

DatetimeIndex(['2019-09-03 12:00:00', '2019-09-04 12:00:00',
               '2019-09-05 12:00:00', '2019-09-06 12:00:00',
               '2019-09-07 12:00:00'],
              dtype='datetime64[ns]', freq='24H')

In [4]:
from_date = '2020-01-01'
to_date = '2020-01-11'
y_max, x_min = -34.675141, 144.652687
y_min, x_max = -38.062278, 151.255470


max_features = 500000
url = f"https://hotspots.dea.ga.gov.au/geoserver/public/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=public:hotspots&outputFormat=application/json&CQL_FILTER=((sensor=%27AVHRR%27%20AND%20(product=%27SRSS%27))%20OR%20(sensor=%27MODIS%27%20AND%20(product=%27MOD14%27))%20OR%20(sensor=%27VIIRS%27%20AND%20(product=%27AFMOD%27%20OR%20product=%27EDR%27)))%20AND%20datetime%20%3E%20%27{from_date}%27%20AND%20datetime%20%3C%20%27{to_date}%27%20AND%20INTERSECTS(location,%20POLYGON(({y_min}%20{x_min},%20{y_min}%20{x_max},%20{y_max}%20{x_max},%20{y_max}%20{x_min},%20{y_min}%20{x_min})))&maxFeatures={max_features}&startIndex=0&sortBy=sensor%20A"
hotspots_extra = gpd.read_file(url)

hotspots_extra = hotspots_extra[hotspots_extra.satellite != 'NOAA 19']
hotspots_extra = hotspots_extra.loc[hotspots_extra.confidence > 49]
hotspots_extra = hotspots_extra[hotspots_extra.within(clip_extent.unary_union)]
hotspots_extra['datetime'] = pd.to_datetime(hotspots_extra['start_dt'])
hotspots_extra['hours_since_hotspot'] = (((hotspots_extra['datetime'].max() - hotspots_extra['datetime'])).astype('timedelta64[m]') / 60)
hotspots_extra = hotspots_extra.loc[:,['datetime', 'latitude', 'longitude', 'hours_since_hotspot', 'confidence', 'geometry']]
hotspots_extra.sort_values('datetime', ascending=True, inplace=True)

NameError: name 'clip_extent' is not defined

In [ ]:
f'POLYGON(({y_min}%20{x_min},%20{y_min}%20{x_max},%20{y_max}%20{x_max},%20{y_max}%20{x_min},%20{y_min}%20{x_min})'

In [64]:
hotspots_subset = pd.concat([hotspots_subset, hotspots_extra])
hotspots_subset.sort_values('datetime', ascending=True, inplace=True)

In [92]:
hotspots_subset.datetime.min()

Timestamp('2019-09-01 00:10:00')

## Iterate times

In [ ]:
comp_dates = pd.date_range(pd.to_datetime('2019-09-03 12:00:00'), 
                           pd.to_datetime('2020-01-07 0:00:00'), 
                           freq='4H')

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
for i, comp_date in enumerate(comp_dates):
    
    print(comp_date)

    hotspots_prev = hotspots_subset.loc[
        hotspots_subset['datetime'] < comp_date].copy()
    hotspots_prev['hours_before'] = ((
        (comp_date - hotspots_prev['datetime'])).astype('timedelta64[m]') / 60)
    hotspots_prev.sort_values('hours_before', inplace=True, ascending=False)
    hotspots_prev = hotspots_prev.loc[hotspots_prev.hours_before < 96]

    ds[['red', 'green', 'blue']].isel(time=0).to_array().plot.imshow(ax=ax, vmax=4500)
    hotspots_prev.plot(ax=ax,
                       column='hours_before',
                       cmap='YlOrRd',
                       markersize=0.5,
                       vmin=0,
                       vmax=96)
    ax.set_facecolor('black')
    ax.set_xlim([144.07, 154.42])
    ax.set_ylim([-39.21, -26.07])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title('');
    ax.text(0.03, 0.95,
            f"{comp_date.tz_localize(tz='UTC').tz_convert('Australia/Sydney'):%Y-%m-%d}",
            ha='left', 
            va='center', 
            transform=ax.transAxes,
           fontdict={'fontsize': 20, 
                     'color': 'white', 
                     'fontname':'Liberation Sans'})
    
    fig.savefig(f'hotspots_{i}.jpeg', 
                bbox_inches='tight',
                dpi=123,
                pad_inches=0)
    plt.cla()


2019-09-03 12:00:00
2019-09-03 16:00:00
2019-09-03 20:00:00
2019-09-04 00:00:00
2019-09-04 04:00:00
2019-09-04 08:00:00
2019-09-04 12:00:00
2019-09-04 16:00:00
2019-09-04 20:00:00
2019-09-05 00:00:00
2019-09-05 04:00:00
2019-09-05 08:00:00
2019-09-05 12:00:00
2019-09-05 16:00:00
2019-09-05 20:00:00
2019-09-06 00:00:00
2019-09-06 04:00:00
2019-09-06 08:00:00
2019-09-06 12:00:00
2019-09-06 16:00:00
2019-09-06 20:00:00
2019-09-07 00:00:00
2019-09-07 04:00:00
2019-09-07 08:00:00
2019-09-07 12:00:00
2019-09-07 16:00:00
2019-09-07 20:00:00
2019-09-08 00:00:00
2019-09-08 04:00:00
2019-09-08 08:00:00
2019-09-08 12:00:00
2019-09-08 16:00:00
2019-09-08 20:00:00
2019-09-09 00:00:00
2019-09-09 04:00:00
2019-09-09 08:00:00
2019-09-09 12:00:00
2019-09-09 16:00:00
2019-09-09 20:00:00
2019-09-10 00:00:00
2019-09-10 04:00:00
2019-09-10 08:00:00
2019-09-10 12:00:00
2019-09-10 16:00:00
2019-09-10 20:00:00
2019-09-11 00:00:00
2019-09-11 04:00:00
2019-09-11 08:00:00
2019-09-11 12:00:00
2019-09-11 16:00:00


In [90]:
!ffmpeg -y -r 24 -i hotspots_%d.jpeg -c:v libx264 -vf crop=720:924:10:10 -pix_fmt yuv420p out.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [ ]:
import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

In [ ]:
comp_dates[0].tz_localize(tz='UTC')

In [ ]:
f"{comp_date.tz_convert('Australia/Sydney'):%Y-%m-%d %H}:00"

In [ ]:
comp_date


stamp = Timestamp('1/1/2014 16:20', tz='America/Sao_Paulo')
   new_stamp = stamp.tz_convert('US/Eastern')

In [ ]:
comp_dates = pd.date_range(pd.to_datetime('2019-09-03 12:00:00'), 
                           pd.to_datetime('2020-01-03 12:00:00'), 
                           freq='24H', tz='UTC')

In [ ]:
comp_dates[0].tz_convert('Australia/Sydney')

In [ ]:
test = hotspots_subset.set_index('datetime').loc['2019-10-05'].cx[148:154.42, -26.07:-30.5]

In [ ]:
test.plot(column='satellite', legend=True)

In [ ]:
-vf fps=25  

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ds[['red', 'green', 'blue']].isel(time=5).to_array().plot.imshow(ax=ax, vmax=3500)
hotspots_prev.plot(ax=ax,
                   column='hours_before',
                   cmap='YlOrRd',
                   markersize=1,
                   vmin=0,
                   vmax=72)
ax.set_facecolor('black')
ax.set_xlim([144.07, 154.42])
ax.set_ylim([-39.21, -26.07])
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.set_title('');

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** October 2019

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)